# Deep-Learning Tensorflow Urban Sound Classification - Sandeep Kanao

Problem Statement
When you start your machine learning journey, you go with simple machine learning problems like titanic survival prediction or digit recogntion. But you still don't have enough practice when it comes to real life problems. To give you a taste of one such problem, we present you "Urban Sound Classification". This practice problem is meant to introduce you to audio processing in the usual classification scenario.

Data
This dataset contains 8732 labeled sound excerpts (<=4s) of urban sounds from 10 classes: air_conditioner, car_horn, children_playing, dog_bark, drilling, enginge_idling, gun_shot, jackhammer, siren, and street_music.

More Details about the contest : https://datahack.analyticsvidhya.com/contest/practice-problem-urban-sound-classification/

Include required libraries

In [174]:
import librosa
import librosa.display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import keras
import os
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix

import numpy as np
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 


Base preparation

In [175]:
test_data_dir = 'Data/test'
train_data_dir = 'Data/train'
train = pd.read_csv(os.path.join(train_data_dir, 'train.csv'))
estimate = pd.read_csv(os.path.join(train_data_dir, 'EstimateTemplate.csv'))
test = pd.read_csv(os.path.join(test_data_dir, 'test.csv'))
def train_parser(row):
    # function to load files and extract features    
    train_file_name = os.path.join(os.path.abspath(train_data_dir), 'Train', str(row.ID) + '.wav')    
    #print(train_file_name)

    # handle exception to check if there isn't a file which is corrupted
    try:
    # here kaiser_fast is a technique used for faster extraction
        X, sample_rate = librosa.load(train_file_name, res_type='kaiser_fast') 
        # we extract mfcc feature from data
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0) 
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None, None

    feature = mfccs
    label = row.Class
    

    return pd.Series([feature, label])

def test_parser(row):
    # function to load files and extract features    
    test_file_name = os.path.join(os.path.abspath(test_data_dir), 'Test', str(row.ID) + '.wav')    
    #print(test_file_name)

    # handle exception to check if there isn't a file which is corrupted
    try:
    # here kaiser_fast is a technique used for faster extraction
        X, sample_rate = librosa.load(test_file_name, res_type='kaiser_fast') 
        # we extract mfcc feature from data
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0) 
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None, None

    feature = mfccs
    label = '0'

    return pd.Series([feature, label])

temp_train = train.apply(train_parser,  axis=1)
temp_test = test.apply(test_parser,  axis=1)
temp_train.columns = ['feature', 'label']
temp_test.columns = ['feature', 'label']
print(temp_train)

                                                feature             label
0     [-82.12358939071989, 139.50591598130987, -42.4...             siren
1     [-15.744005405358058, 124.1199599305049, -29.4...      street_music
2     [-123.39365145003913, 15.181946313102896, -50....          drilling
3     [-213.27878814908152, 89.32358896182456, -55.2...             siren
4     [-237.92647882472895, 135.90246127730546, 39.2...          dog_bark
5     [-250.35704278618198, 128.01181557087781, -47....  children_playing
6     [-203.97553041599923, 83.95814057767474, -66.7...      street_music
7     [-55.40055461612478, 22.767318943295965, -46.0...          drilling
8     [-110.22915733719984, 91.25898699199308, -137....          gun_shot
9     [-337.7600192890491, 140.77517821184472, -15.5...          dog_bark
10    [-182.77830877917827, 168.22353311947717, 19.1...     engine_idling
11    [-178.8089051222893, 174.4008742877636, -30.28...             siren
12    [-542.225972382311, 21.421938715

Compile model

In [199]:
epochs = 2000
batch_size = 64

from sklearn.preprocessing import LabelEncoder

train_X = np.array(temp_train.feature.tolist())
train_y = np.array(temp_train.label.tolist())
test_X = np.array(temp_test.feature.tolist())
test_y = np.array(temp_test.label.tolist())

lb = LabelEncoder()
print(train_y)
print(lb.fit_transform(train_y))

label_mapping = dict(zip(lb.fit_transform(train_y), train_y))
print(label_mapping)

train_y = np_utils.to_categorical(lb.fit_transform(train_y))
test_y = np_utils.to_categorical(lb.fit_transform(test_y))


num_labels = train_y.shape[1]
filter_size = 2

# build model
model = Sequential()

model.add(Dense(256, input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.001))
model.add(Dense(num_labels))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

model.fit(train_X, train_y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

['siren' 'street_music' 'drilling' 'siren' 'dog_bark' 'children_playing'
 'street_music' 'drilling' 'gun_shot' 'dog_bark' 'engine_idling' 'siren'
 'gun_shot' 'engine_idling' 'air_conditioner' 'air_conditioner'
 'engine_idling' 'siren' 'children_playing' 'jackhammer' 'gun_shot'
 'siren' 'jackhammer' 'street_music' 'jackhammer' 'engine_idling' 'siren'
 'children_playing' 'jackhammer' 'engine_idling' 'air_conditioner'
 'car_horn' 'drilling' 'siren' 'gun_shot' 'children_playing'
 'children_playing' 'drilling' 'gun_shot' 'siren' 'car_horn' 'drilling'
 'street_music' 'jackhammer' 'children_playing' 'dog_bark' 'dog_bark'
 'street_music' 'street_music' 'jackhammer' 'air_conditioner' 'dog_bark'
 'jackhammer' 'gun_shot' 'jackhammer' 'street_music' 'car_horn'
 'street_music' 'children_playing' 'car_horn' 'siren' 'jackhammer' 'siren'
 'street_music' 'dog_bark' 'jackhammer' 'air_conditioner' 'drilling'
 'air_conditioner' 'jackhammer' 'street_music' 'street_music'
 'air_conditioner' 'drilling' 'engi

Train on 4348 samples, validate on 1087 samples
Epoch 1/2000
4348/4348 [==============================] - 1s 154us/step - loss: 8.1098 - acc: 0.3257 - val_loss: 6.4568 - val_acc: 0.3983
Epoch 2/2000
4348/4348 [==============================] - 0s 35us/step - loss: 6.1606 - acc: 0.4623 - val_loss: 6.1774 - val_acc: 0.4710
Epoch 3/2000
4348/4348 [==============================] - 0s 34us/step - loss: 5.9984 - acc: 0.4959 - val_loss: 6.0353 - val_acc: 0.5005
Epoch 4/2000
4348/4348 [==============================] - 0s 35us/step - loss: 5.8887 - acc: 0.5106 - val_loss: 5.9537 - val_acc: 0.5170
Epoch 5/2000
4348/4348 [==============================] - 0s 34us/step - loss: 5.8423 - acc: 0.5225 - val_loss: 5.9353 - val_acc: 0.5078
Epoch 6/2000
4348/4348 [==============================] - 0s 34us/step - loss: 5.8497 - acc: 0.5267 - val_loss: 5.9471 - val_acc: 0.5087
Epoch 7/2000
4348/4348 [==============================] - 0s 34us/step - loss: 5.7443 - acc: 0.5547 - val_loss: 5.8609 - val_acc:

4348/4348 [==============================] - 0s 40us/step - loss: 5.4447 - acc: 0.6474 - val_loss: 5.7812 - val_acc: 0.5851
Epoch 60/2000
4348/4348 [==============================] - 0s 34us/step - loss: 5.4512 - acc: 0.6467 - val_loss: 5.8479 - val_acc: 0.5777
Epoch 61/2000
4348/4348 [==============================] - 0s 34us/step - loss: 5.4497 - acc: 0.6470 - val_loss: 5.7280 - val_acc: 0.5952
Epoch 62/2000
4348/4348 [==============================] - 0s 35us/step - loss: 5.4301 - acc: 0.6546 - val_loss: 5.9124 - val_acc: 0.5750
Epoch 63/2000
4348/4348 [==============================] - 0s 35us/step - loss: 5.4261 - acc: 0.6564 - val_loss: 5.7311 - val_acc: 0.6072
Epoch 64/2000
4348/4348 [==============================] - 0s 37us/step - loss: 5.4167 - acc: 0.6587 - val_loss: 5.7554 - val_acc: 0.6017
Epoch 65/2000
4348/4348 [==============================] - 0s 35us/step - loss: 5.4071 - acc: 0.6635 - val_loss: 5.7578 - val_acc: 0.5980
Epoch 66/2000
4348/4348 [=======================

4348/4348 [==============================] - 0s 37us/step - loss: 1.8718 - acc: 0.8836 - val_loss: 2.2616 - val_acc: 0.8206
Epoch 119/2000
4348/4348 [==============================] - 0s 34us/step - loss: 1.8744 - acc: 0.8820 - val_loss: 2.3025 - val_acc: 0.7985
Epoch 120/2000
4348/4348 [==============================] - 0s 34us/step - loss: 1.8815 - acc: 0.8790 - val_loss: 2.3435 - val_acc: 0.8151
Epoch 121/2000
4348/4348 [==============================] - 0s 34us/step - loss: 1.8978 - acc: 0.8744 - val_loss: 2.2828 - val_acc: 0.8142
Epoch 122/2000
4348/4348 [==============================] - 0s 34us/step - loss: 1.9204 - acc: 0.8659 - val_loss: 2.3357 - val_acc: 0.7948
Epoch 123/2000
4348/4348 [==============================] - 0s 35us/step - loss: 1.9188 - acc: 0.8703 - val_loss: 2.4900 - val_acc: 0.7737
Epoch 124/2000
4348/4348 [==============================] - 0s 35us/step - loss: 1.8955 - acc: 0.8735 - val_loss: 2.3376 - val_acc: 0.8068
Epoch 125/2000
4348/4348 [================

4348/4348 [==============================] - 0s 37us/step - loss: 0.1375 - acc: 0.9890 - val_loss: 0.6587 - val_acc: 0.8951
Epoch 178/2000
4348/4348 [==============================] - 0s 34us/step - loss: 0.1319 - acc: 0.9910 - val_loss: 0.6683 - val_acc: 0.8988
Epoch 179/2000
4348/4348 [==============================] - 0s 34us/step - loss: 0.1272 - acc: 0.9924 - val_loss: 0.6694 - val_acc: 0.8960
Epoch 180/2000
4348/4348 [==============================] - 0s 33us/step - loss: 0.1274 - acc: 0.9920 - val_loss: 0.6832 - val_acc: 0.8997
Epoch 181/2000
4348/4348 [==============================] - 0s 32us/step - loss: 0.1267 - acc: 0.9924 - val_loss: 0.6654 - val_acc: 0.8942
Epoch 182/2000
4348/4348 [==============================] - 0s 43us/step - loss: 0.1258 - acc: 0.9924 - val_loss: 0.6795 - val_acc: 0.8970
Epoch 183/2000
4348/4348 [==============================] - 0s 36us/step - loss: 0.1309 - acc: 0.9915 - val_loss: 0.7129 - val_acc: 0.8933
Epoch 184/2000
4348/4348 [================

...

4348/4348 [==============================] - 0s 36us/step - loss: 0.1224 - acc: 0.9924 - val_loss: 1.0456 - val_acc: 0.9062
Epoch 1758/2000
4348/4348 [==============================] - 0s 34us/step - loss: 0.1224 - acc: 0.9924 - val_loss: 1.0435 - val_acc: 0.9071
Epoch 1759/2000
4348/4348 [==============================] - 0s 35us/step - loss: 0.1223 - acc: 0.9924 - val_loss: 1.0427 - val_acc: 0.9071
Epoch 1760/2000
4348/4348 [==============================] - 0s 34us/step - loss: 0.1245 - acc: 0.9917 - val_loss: 1.0484 - val_acc: 0.9052
Epoch 1761/2000
4348/4348 [==============================] - 0s 34us/step - loss: 0.1256 - acc: 0.9913 - val_loss: 1.1196 - val_acc: 0.8988
Epoch 1762/2000
4348/4348 [==============================] - 0s 34us/step - loss: 0.1261 - acc: 0.9917 - val_loss: 1.1038 - val_acc: 0.8942
Epoch 1763/2000
4348/4348 [==============================] - 0s 35us/step - loss: 0.1226 - acc: 0.9924 - val_loss: 1.0680 - val_acc: 0.9052
Epoch 1764/2000
4348/4348 [=========

4348/4348 [==============================] - 0s 37us/step - loss: 0.1351 - acc: 0.9903 - val_loss: 1.1307 - val_acc: 0.9016
Epoch 1816/2000
4348/4348 [==============================] - 0s 35us/step - loss: 0.1261 - acc: 0.9910 - val_loss: 1.1221 - val_acc: 0.8914
Epoch 1817/2000
4348/4348 [==============================] - 0s 34us/step - loss: 0.1262 - acc: 0.9910 - val_loss: 1.1358 - val_acc: 0.8960
Epoch 1818/2000
4348/4348 [==============================] - 0s 35us/step - loss: 0.1360 - acc: 0.9897 - val_loss: 1.1500 - val_acc: 0.8970
Epoch 1819/2000
4348/4348 [==============================] - 0s 35us/step - loss: 0.1235 - acc: 0.9917 - val_loss: 1.1101 - val_acc: 0.9025
Epoch 1820/2000
4348/4348 [==============================] - 0s 34us/step - loss: 0.1238 - acc: 0.9920 - val_loss: 1.1445 - val_acc: 0.8988
Epoch 1821/2000
4348/4348 [==============================] - 0s 34us/step - loss: 0.1224 - acc: 0.9924 - val_loss: 1.1285 - val_acc: 0.8997
Epoch 1822/2000
4348/4348 [=========

4348/4348 [==============================] - 0s 37us/step - loss: 0.1264 - acc: 0.9908 - val_loss: 1.2882 - val_acc: 0.8813
Epoch 1874/2000
4348/4348 [==============================] - 0s 34us/step - loss: 0.1276 - acc: 0.9915 - val_loss: 1.1636 - val_acc: 0.9016
Epoch 1875/2000
4348/4348 [==============================] - 0s 36us/step - loss: 0.1233 - acc: 0.9922 - val_loss: 1.1351 - val_acc: 0.8924
Epoch 1876/2000
4348/4348 [==============================] - 0s 35us/step - loss: 0.1230 - acc: 0.9922 - val_loss: 1.1445 - val_acc: 0.8933
Epoch 1877/2000
4348/4348 [==============================] - 0s 35us/step - loss: 0.1313 - acc: 0.9915 - val_loss: 1.1787 - val_acc: 0.8896
Epoch 1878/2000
4348/4348 [==============================] - 0s 35us/step - loss: 0.1225 - acc: 0.9924 - val_loss: 1.1388 - val_acc: 0.8933
Epoch 1879/2000
4348/4348 [==============================] - 0s 36us/step - loss: 0.1248 - acc: 0.9920 - val_loss: 1.0675 - val_acc: 0.8997
Epoch 1880/2000
4348/4348 [=========

4348/4348 [==============================] - 0s 37us/step - loss: 0.1224 - acc: 0.9924 - val_loss: 1.1698 - val_acc: 0.8951
Epoch 1932/2000
4348/4348 [==============================] - 0s 34us/step - loss: 0.1261 - acc: 0.9922 - val_loss: 1.1680 - val_acc: 0.8970
Epoch 1933/2000
4348/4348 [==============================] - 0s 35us/step - loss: 0.1229 - acc: 0.9920 - val_loss: 1.1573 - val_acc: 0.8905
Epoch 1934/2000
4348/4348 [==============================] - 0s 34us/step - loss: 0.1288 - acc: 0.9913 - val_loss: 1.1012 - val_acc: 0.9006
Epoch 1935/2000
4348/4348 [==============================] - 0s 36us/step - loss: 0.1303 - acc: 0.9903 - val_loss: 1.1078 - val_acc: 0.8997
Epoch 1936/2000
4348/4348 [==============================] - 0s 35us/step - loss: 0.1334 - acc: 0.9906 - val_loss: 1.1782 - val_acc: 0.8970
Epoch 1937/2000
4348/4348 [==============================] - 0s 36us/step - loss: 0.1282 - acc: 0.9910 - val_loss: 1.2199 - val_acc: 0.8914
Epoch 1938/2000
4348/4348 [=========

4348/4348 [==============================] - 0s 39us/step - loss: 0.1248 - acc: 0.9920 - val_loss: 1.1558 - val_acc: 0.8970
Epoch 1990/2000
4348/4348 [==============================] - 0s 36us/step - loss: 0.1236 - acc: 0.9920 - val_loss: 1.1651 - val_acc: 0.8970
Epoch 1991/2000
4348/4348 [==============================] - 0s 39us/step - loss: 0.1510 - acc: 0.9864 - val_loss: 1.2399 - val_acc: 0.8960
Epoch 1992/2000
4348/4348 [==============================] - 0s 36us/step - loss: 0.1389 - acc: 0.9890 - val_loss: 1.1731 - val_acc: 0.8970
Epoch 1993/2000
4348/4348 [==============================] - 0s 35us/step - loss: 0.1356 - acc: 0.9890 - val_loss: 1.2058 - val_acc: 0.8924
Epoch 1994/2000
4348/4348 [==============================] - 0s 37us/step - loss: 0.1445 - acc: 0.9887 - val_loss: 1.2088 - val_acc: 0.9016
Epoch 1995/2000
4348/4348 [==============================] - 0s 38us/step - loss: 0.1276 - acc: 0.9913 - val_loss: 1.1920 - val_acc: 0.8960
Epoch 1996/2000
4348/4348 [=========

Save Model

In [201]:
model_name = 'Speech_Detection_Model.h5'
save_dir = os.path.join(os.getcwd(), 'saved_models')
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at C:\MusicalInstrumentDetection\saved_models\Speech_Detection_Model.h5 


In [202]:
import json
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

Load model and get precision

In [203]:
opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)
# loading json and creating model
from keras.models import model_from_json
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("saved_models/Speech_Detection_Model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
score = loaded_model.evaluate(train_X, train_y, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk
acc: 97.55%


In [204]:
preds = loaded_model.predict(test_X, 
                         batch_size=32, 
                         verbose=1)



3297/3297 [==============================] - 0s 59us/step


In [205]:
#preds
#print(temp_train) 
#print(train_y)

temp_train.label.value_counts(normalize=True)

jackhammer          0.122907
engine_idling       0.114811
siren               0.111684
air_conditioner     0.110396
dog_bark            0.110396
street_music        0.110396
drilling            0.110396
children_playing    0.110396
car_horn            0.056302
gun_shot            0.042318
Name: label, dtype: float64

In [207]:
#temp_train['Class'] = 'siren'
#temp_train.to_csv('test_train.csv', index=False)
submit_test = pd.read_csv(os.path.join(test_data_dir, 'submittemplate.csv'))

pred = model.predict_classes(test_X)
submit_test['Class'] = pred
print(submit_test)
submit_test.to_csv('sub04.csv', index=False)

#print(temp_train) 
#print(train_y)
#print(temp_train)
#print(temp_test)

pred = model.predict_classes(train_X)
#print(pred)
#print(train['Class'])
train['Class'] = pred
#estimate = np.array([])

#train['Label'] = label_mapping[pred]
for label in pred :  
    #print(label_mapping[label])
    #estimate['Estimate'] = label_mapping[label]
    estimate = np.append(estimate, label_mapping[label])
print(estimate)   
train.to_csv('train_sub04.csv', index=False)

      Class    ID
0         4     5
1         3     7
2         4     8
3         3     9
4         5    13
5         4    14
6         2    16
7         2    21
8         8    23
9         8    25
10        3    28
11        3    29
12        3    30
13        6    31
14        9    34
15        0    39
16        3    41
17        2    51
18        9    53
19        9    55
20        4    58
21        3    62
22        8    64
23        9    69
24        4    70
25        2    73
26        2    75
27        2    85
28        2    86
29        3    89
...     ...   ...
3267      4  8642
3268      9  8645
3269      7  8647
3270      1  8648
3271      8  8652
3272      4  8655
3273      5  8661
3274      3  8662
3275      2  8668
3276      3  8669
3277      4  8671
3278      0  8672
3279      3  8674
3280      5  8680
3281      0  8683
3282      4  8686
3283      9  8687
3284      5  8688
3285      2  8689
3286      3  8692
3287      2  8696
3288      8  8697
3289      2  8698
3290      

Map labels with actual class to get the actual value